This notebook clean the OG Rust(1987) data in ACSII. The code is borrowed from https://notes.quantecon.org/submission/6234fe0f96e1ce001b61fad8.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.optimize import minimize, fixed_point
from numba import njit, float64, float32, int64, prange
from numba.experimental import jitclass


In [2]:
# Import the raw data
b1_raw = pd.read_csv(r'.\OpenSourceEconomics-zurcher-data-43d37bd\data\original_data\g870.asc', header=None)
b1_raw = pd.DataFrame(b1_raw.values.reshape((15, int(len(b1_raw) / 15)))).T

b2_raw = pd.read_csv(r'.\OpenSourceEconomics-zurcher-data-43d37bd\data\original_data\rt50.asc', header=None)
b2_raw = pd.DataFrame(b2_raw.values.reshape((4, int(len(b2_raw) / 4)))).T

b3_raw = pd.read_csv(r'.\OpenSourceEconomics-zurcher-data-43d37bd\data\original_data\t8h203.asc', header=None)
b3_raw = pd.DataFrame(b3_raw.values.reshape((48, int(len(b3_raw) / 48)))).T

b4_raw = pd.read_csv(r'.\OpenSourceEconomics-zurcher-data-43d37bd\data\original_data\a530875.asc', header=None)
b4_raw = pd.DataFrame(b4_raw.values.reshape((37, int(len(b4_raw) / 37)))).T

In [3]:
def extract_info(b):

    # Create separate dataframe for bus info
    new_index = ['' for x in range(11)]
    new_index[0] = 'bus_number'
    new_index[1] = 'month_purchased'
    new_index[2] = 'year_purchased'
    new_index[3] = 'month_replacement_1'
    new_index[4] = 'year_replacement_1'
    new_index[5] = 'odometer_replacement_1'
    new_index[6] = 'month_replacement_2'
    new_index[7] = 'year_replacement_2'
    new_index[8] = 'odometer_replacement_2'
    new_index[9] = 'month_begin'
    new_index[10] = 'year_begin'
    b_info = b.iloc[:11]
    b_info.index = new_index
    b_info = b_info.T.set_index('bus_number').T
    
    # Set bus number to be column headers
    b = b.T.set_index([0]).T

    # Drop bus info
    b = b[10:]

    # Add dates to row index
    month = b_info.loc["month_begin"].values[0]
    year = b_info.loc["year_begin"].values[0]
    b.index = pd.date_range(f"{year}-{month}-01", periods=len(b), freq="MS")

    # Fix month and years of odometer replacements to be date corresponding to odometer replacement
    # Rust points out this differs in some cases
    col_map = pd.DataFrame(b.columns).to_dict()[0]
    for o in ['1', '2']:
        x, y = np.where(b > b_info.loc[f'odometer_replacement_{o}', :])
        replacement = pd.DataFrame([x, y], index=['row', 'column']).T
        replacement = replacement.groupby('column').min()
        replacement.index = replacement.index.map(col_map)
        replacement[f'month_{o}'] = replacement['row'].apply(lambda x: b.index[x].month)
        replacement[f'year_{o}'] = replacement['row'].apply(lambda x: np.mod(b.index[x].year, 1900))
        replacement[f'replace_{o}'] = replacement['row'].apply(lambda x: pd.Timestamp(year=b.index[x].year, month=b.index[x].month, day=1))
        replacement[replacement.row == 0] = 0
        # Append checks to dataframe
        replacement = replacement.drop('row', axis=1)
        b_info = pd.concat([b_info, replacement.T])

    return b_info, b

In [4]:
def update_b(b, b_info):
    '''Resets mileage after replacement and returns investment decision matrix'''

    b_reset = b.copy()
    b_i = b.copy()
    b_i[:] = 0

    for col in b.iteritems():

        bus_number = col[0]
        bus_info = b_info[bus_number].copy()

        for r in ('1'):
            replace_date = bus_info[f'replace_{r}']

            if replace_date != 0:

                if r == '1':
                    odometer_replacement = b.loc[replace_date, bus_number]

                if r == '2':
                    # Subtract mileage from earlier replacement
                    replace_date_1 = bus_info[f'replace_1']
                    previous_mileage = b.loc[replace_date_1, bus_number]
                    odometer_replacement = b.loc[replace_date, bus_number] - previous_mileage
                
                bus_odometer = b_reset[bus_number].copy()
                
                # Find replacement date then subtract odometer replacement value from
                # odometer readings after replacement
                bus_odometer[bus_odometer.index > replace_date] -= odometer_replacement
                b_reset[bus_number] = bus_odometer

                # Set decision = 1 on replacement date
                b_i.loc[replace_date, bus_number] = 1

    return b, b_reset, b_i

In [5]:
def discretize(b, d=5000):
    '''Discretizes odometer data into buckets of length d'''

    return np.floor(b / d)

In [6]:
# Extract information from the raw datasets
b1_info, b1 = extract_info(b1_raw)
b2_info, b2 = extract_info(b2_raw)
b3_info, b3 = extract_info(b3_raw)
b4_info, b4 = extract_info(b4_raw)

# Get relevant data and merge
b1, b1_reset, b1_i = update_b(b1, b1_info)
b2, b2_reset, b2_i = update_b(b2, b2_info)
b3, b3_reset, b3_i = update_b(b3, b3_info)
b4, b4_reset, b4_i = update_b(b4, b4_info)

b = pd.concat([b1, b2, b3, b4], axis=1, join='outer')
b_reset = pd.concat([b1_reset, b2_reset, b3_reset, b4_reset], axis=1, join='outer')
b_i = pd.concat([b1_i, b2_i, b3_i, b4_i], axis=1, join='outer')

# Create dataframe for likelihood estimation
b_data = pd.concat([discretize(b_reset.T.stack()), b_i.T.stack()], axis=1)
b_data.columns = ['state', 'decision']

In [7]:
b_data['next_state'] = b_data['state'].shift(-1)
b_data['change'] = b_data['next_state'] - b_data['state']

In [58]:
b_data.to_csv('bus_data.csv', index=False)

In [8]:
spec = [('β', float64),
        ('θ_1', float64),
        ('RC', float64),
        ('T', int64),
        ('p', float64),
        ('q', float64),
        ('scale', float64),
        ('P', float64[:, :]),
        ('x', float64[:])]

@jitclass(spec)
class Rust(object):

    def __init__(self, β=0.9999, θ_1=3.6, 
                 RC=10, T=90, p=0.3497, 
                 q=0.6387, scale=0.001):

        self.β = β
        self.θ_1 = θ_1
        self.RC = RC
        self.T = T
        self.p = p
        self.q = q
        self.scale = scale

        # Construct transition matrix
        P = np.zeros((T, T))
        np.fill_diagonal(P, p)
        P += np.diag(np.ones(T - 1) * q, 1)
        P += np.diag(np.ones(T - 2) * (1 - p - q), 2)
        P[:, -1] += 1 - P.sum(1)  # Adjust to sum to 1
        self.P = P

        # Statespace of x
        self.x = np.arange(T, dtype=np.float64)

In [9]:

@njit
def c(x, rust):
    '''Linear cost function'''

    scale = rust.scale
    θ_1 = rust.θ_1

    return scale * θ_1 * x

@njit
def u(x, i, rust):
    '''Utility function'''
    
    if i == 0:
        return -c(x, rust)

    if i == 1:
        return -c(x, rust) - rust.RC

In [10]:
@njit
def solve_EV(rust, tol=1e-3, maxiter=300000, print_flag=False):

    T = rust.T
    β = rust.β
    P = np.ascontiguousarray(rust.P)
    x = rust.x

    # Initial guess of value function
    EV = np.zeros(T)

    # Bellman operator
    def T(EV):
        wait = u(x, 0, rust) + β * EV
        replace = u(x[0], 1, rust) + β * EV[0]
        EV_new = np.exp(replace - EV) + np.exp(wait - EV)
        EV_new = P @ (np.log(EV_new) + EV)
        return EV_new

    # Find the fixed point of Bellman operator
    i = 0
    dist = 1e3
    for i in prange(maxiter):
        EV_new = T(EV)
        dist = np.linalg.norm(EV - EV_new)

        if dist < tol:
            if print_flag:
                print(f'Converged in {i} iterations')
            return EV
        else:
            i += 1
            EV = EV_new.copy()
    
    if print_flag:
        print('Reached max iterations')
        
    return EV

In [11]:
# Discretize the odometer readings
b_discrete = discretize(b)

# Calculate the states d = {0, 1, 2}
b_state = b_discrete - b_discrete.shift(1)

# Find the transition probabilities
N = b_state.notna().sum().sum()
p = (b_state == 0).sum().sum() / N
q = (b_state == 1).sum().sum() / N
(p, q)

(0.35102991662579697, 0.6374448258950466)

In [12]:
r = Rust(p=p, q=q)
EV_star = solve_EV(r)

plt.plot(r.x, EV_star, label=rf'$\beta = {r.β}$')

plt.title('Expected value function')
plt.xlabel('$x$')
plt.legend()
plt.show()

In [13]:
@njit
def conditional_probability(rust):

    β = rust.β
    x = rust.x
    P = rust.P

    # Solve inner loop
    EV = solve_EV(rust)

    wait = u(x, 0, rust) + β * P @ EV
    replace = u(x[0], 1, rust) + β * EV[0]
    P_wait = 1 / (1 + np.exp(replace - wait))
    P_replace = 1 - P_wait
    
    return P_wait, P_replace

In [14]:
def log_likelihood(θ,             # Parameters to search over 
                   b_data,        # Data consists of states and decisions
                   p,             # Absorbing state probability
                   q):            # Transition state 1 probability

    # Guess of parameters
    θ_1, RC = θ

    # Set up model with guess
    r = Rust(θ_1=θ_1, RC=RC, p=p, q=q)

    # Solve inner loop and find choice
    # probabilities conditional on x and i
    P_wait, P_replace = conditional_probability(r)

    # Log likelihood is sum of conditional probabilities given observed data
    logL = 0
    for decision, state in zip(b_data.decision, b_data.state):
        state = int(state)

        # Wait
        if decision == 0:
            logL += np.log(P_wait[state])
        
        # Replace
        if decision == 1:
            logL += np.log(P_replace[state])

    return -logL

In [15]:
%%time
θ_star = minimize(log_likelihood, x0=(0.1, 10), args=(b_data, p, q)).x

CPU times: total: 37.3 s
Wall time: 1min 22s


In [17]:
r.x

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89.])